
### **This notebook shows how to use a .fchk file to run an minimal basis iterative stockholder (MBIS) partitioning using the IOData, Grid, GBasis, and DensPart packages.**


In [ ]:
# Install packages in Google Colab. Don't run this cell if packages/data is already in your environment
! pip install scipy==1.10.1
! pip install git+https://github.com/theochem/iodata.git
! pip install git+https://github.com/theochem/grid.git
! pip install git+https://github.com/theochem/gbasis.git
! pip install git+https://github.com/theochem/denspart.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.2
    Uninstalling scipy-1.11.2:
      Successfully uninstalled scipy-1.11.2
  Cloning https://github.com/theochem/iodata.git to /tmp/pip-req-build-807wolw7
  Running command git clone --filter=blob:none --quiet https://github.com/theochem/iodata.git /tmp/pip-req-build-807wolw7
  Resolved https://github.com/theochem/iodata.git to commit ca6113b60fc967f3c280bd03374aaba1644d672e
  Preparing metadata (setup.py) ... done
  Created wheel for qc-iodata: filename=qc_iodata-0.0.0.post0-py3-none-any.whl size=3337351 sha256=000285707cb1fda71814f3146b8c456f45e6792ce12d1fd3b9fe99c78a62fedf
  Stored in directory: /tmp/pip-ephem-wheel-cache-695ad14x/wheels/c2/0a/62/493efe62d77a51ee3d7918ade8cfe73f1d18416f9679edd471
Successfully built qc-iodata
  Cloning https://github.com/theochem/grid.git to /tmp/pip-req-build-wzneb0kj
  Running command git clone

In [ ]:
import glob
import csv
from iodata import load_one
from grid.becke import BeckeWeights
from grid.molgrid import MolGrid
from grid.onedgrid import GaussChebyshev
from grid.rtransform import BeckeRTransform
from gbasis.wrappers import from_iodata
from gbasis.evals.density import evaluate_density
import numpy as np
from denspart.mbis import MBISProModel
from denspart.vh import optimize_reduce_pro_model

# Define a directory containing your fchk files
fchk_directory = "/content/"

fchk_files = glob.glob(fchk_directory + "*.fchk")

# Create a list to store files that did not converge
did_not_converge = []

with open("atomic_charges.csv", mode="w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["File", "Atomic Charges"])
    for fchk_file in fchk_files:
        print("Processing file:", fchk_file)

        try:
            # Load the fchk file using IOData
            mol = load_one(fchk_file)

            # Make Becke-Lebedev molecular grid (using preset grid)
            oned = GaussChebyshev(100)
            rgrid = BeckeRTransform(1e-4, 1.5).transform_1d_grid(oned)
            grid = MolGrid.from_preset(mol.atnums, mol.atcoords, rgrid, "ultrafine", BeckeWeights())

            # Calculate the one-electron density matrix
            one_rdm = np.dot(mol.mo.coeffs * mol.mo.occs, mol.mo.coeffs.T)

            # Convert the basis from IOData format
            basis = from_iodata(mol)

            # Evaluate the density
            density = evaluate_density(one_rdm, basis[0], grid.points, coord_type=basis[1])

            # Create and optimize the MBISProModel
            pro_model_init = MBISProModel.from_geometry(mol.atnums, mol.atcoords)
            pro_model, localgrids = optimize_reduce_pro_model(pro_model_init, grid, density)

            # Extract atomic charges
            charges = pro_model.charges

            writer.writerow([fchk_file, charges])
        except RuntimeError as e:
            print(f"Optimization for {fchk_file} failed with error: {str(e)}")
            did_not_converge.append(fchk_file)

print("Results saved to atomic_charges.csv")

# Print the files that did not converge
if did_not_converge:
    print("Files that did not converge:")
    for file in did_not_converge:
        print(file)
else:
    print("All files converged successfully.")


Processing file: /content/0002_q000_m01_k00_force_uhf_def2qzvpd.fchk


/usr/local/lib/python3.10/dist-packages/grid/atomgrid.py:889: UserWarning: Lebedev weights are negative which can introduce round-off errors.
  sphere_grid = AngularGrid(degree=deg_i, use_spherical=use_spherical)


Building local grids
Integral of density: 24.000077707552343
Optimization
#Iter  #Call         ekld          kld  -constraint     grad.rms  cputime (s)
-----  -----  -----------  -----------  -----------  -----------  -----------
    1      1    0.6610780    0.6611557  -7.7708e-05   1.7120e-01    0.1140468
    2      2    0.4594864    0.3550240   1.0446e-01   9.7573e-02    0.2903631
    3      3    6.0447939    4.1388270   1.9060e+00   1.7708e+00    0.2896447
    4      4    0.5558312   -0.2460909   8.0192e-01   1.4101e-01    0.2866000
    5      6    2.6180010    5.3361756  -2.7182e+00   4.2688e-01    0.2779580
    6      8    1.2156402    1.7862915  -5.7065e-01   5.0562e-01    0.2794995
    7     10    0.5975319    0.8672124  -2.6968e-01   3.0283e-01    0.2836473
    8     11    0.3827099   -0.1191500   5.0186e-01   1.5072e-01    0.2737213
    9     13    1.5503360    0.8004241   7.4991e-01   3.8765e-01    0.2890754
   10     15    0.4644326   -0.3589278   8.2336e-01   1.5451e-01    

/usr/local/lib/python3.10/dist-packages/grid/atomgrid.py:889: UserWarning: Lebedev weights are negative which can introduce round-off errors.
  sphere_grid = AngularGrid(degree=deg_i, use_spherical=use_spherical)


Building local grids
Integral of density: 24.000019876660797
Optimization
#Iter  #Call         ekld          kld  -constraint     grad.rms  cputime (s)
-----  -----  -----------  -----------  -----------  -----------  -----------
    1      1    0.7690427    0.7690626  -1.9877e-05   1.7107e-01    0.2485953
    2      2    0.5430706    0.4221732   1.2090e-01   1.0396e-01    0.3288326
    3      3    6.6462285    4.7270348   1.9192e+00   2.1915e+00    0.3061495
    4      4    0.6276685   -0.7601807   1.3878e+00   1.3951e-01    0.2648097
    5      6    3.8213553    5.0261098  -1.2048e+00   1.4387e+00    0.3060322
    6      7    0.4069533   -0.6742324   1.0812e+00   1.0078e-01    0.3210990
    7      8    0.3752289   -0.6536876   1.0289e+00   8.3876e-02    0.3056727
    8      8    0.3752289   -0.6536876   1.0289e+00   8.3876e-02    0.3056727
    9     10    0.3677257    0.3348271   3.2899e-02   8.4855e-02    0.3222120
   10     12    1.1194520    2.0132026  -8.9375e-01   8.0767e-01    

...

Building local grids
Integral of density: 24.000017159892156
Optimization
#Iter  #Call         ekld          kld  -constraint     grad.rms  cputime (s)
-----  -----  -----------  -----------  -----------  -----------  -----------
    1      1    0.7635253    0.7635424  -1.7160e-05   1.6985e-01    0.2465771
    2      2    0.5414435    0.4174310   1.2401e-01   1.0283e-01    0.2554057
    3      3    6.8039831    4.8116064   1.9924e+00   2.2484e+00    0.3277873
    4      4    0.6366136   -0.7751154   1.4117e+00   1.4015e-01    0.3166785
    5      5    3.9671746    5.1966079  -1.2294e+00   1.5054e+00    0.3136287
    6      6    0.4137148   -0.6933521   1.1071e+00   1.0337e-01    0.3207777
    7      7    0.3816591   -0.6731713   1.0548e+00   8.4519e-02    0.2910740
    8      7    0.3816591   -0.6731713   1.0548e+00   8.4519e-02    0.2910740
    9      9    0.3869161    0.4570937  -7.0178e-02   8.8117e-02    0.3005358
   10     10    0.2823639   -0.1856013   4.6797e-01   3.5355e-02    

/usr/local/lib/python3.10/dist-packages/grid/atomgrid.py:889: UserWarning: Lebedev weights are negative which can introduce round-off errors.
  sphere_grid = AngularGrid(degree=deg_i, use_spherical=use_spherical)


Building local grids
Integral of density: 24.00001848931206
Optimization
#Iter  #Call         ekld          kld  -constraint     grad.rms  cputime (s)
-----  -----  -----------  -----------  -----------  -----------  -----------
    1      1    0.6930717    0.6930902  -1.8489e-05   1.7082e-01    0.1651423
    2      2    0.4800832    0.3529466   1.2714e-01   9.9069e-02    0.3246532
    3      3    7.2743976    5.0590839   2.2153e+00   2.5553e+00    0.2920507
    4      4    0.5882964   -0.8947585   1.4831e+00   1.4062e-01    0.3149627
    5      5    0.5205766   -0.7329609   1.2535e+00   1.3238e-01    0.3156234
    6      6    0.3452904   -0.1386992   4.8399e-01   1.0221e-01    0.3014057
    7      7    0.3418898   -0.1450254   4.8692e-01   9.9294e-02    0.3056463
    8      8    0.2928985   -0.2731399   5.6604e-01   6.4577e-02    0.3108283
    9      8    0.2928985   -0.2731399   5.6604e-01   6.4577e-02    0.3108283
   10      9    2.8848382    5.1874786  -2.3026e+00   1.7325e+00    0

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


   18     21    0.2797653    0.3282199  -4.8455e-02   1.2021e-01    0.3131444
   19     22    0.2319492    0.1511931   8.0756e-02   5.2318e-02    0.2994748
   20     23    0.2245970    0.0807421   1.4385e-01   1.9650e-02    0.3354283
   21     25    0.3086153    0.1837158   1.2490e-01   1.3050e-01    0.3044608
   22     26    0.2215109    0.1218859   9.9625e-02   1.9770e-02    0.2882819
   23     27    0.2213625    0.0614505   1.5991e-01   1.4798e-02    0.3188774
   24     27    0.2213625    0.0614505   1.5991e-01   1.4798e-02    0.3188774
   25     28    0.2184244    0.1912532   2.7171e-02   4.8803e-03    0.3148350
   26     30    1.2410085    0.3007290   9.4028e-01   4.6402e-01    0.2962521
   27     32    0.2211017   -0.0595201   2.8062e-01   1.5372e-02    0.2944528
   28     33    0.2181605    0.1830127   3.5148e-02   1.1509e-02    0.3207826
   29     34    0.2176021    0.1768081   4.0794e-02   5.2838e-03    0.3028317
   30     36    0.2238557    0.1062198   1.1764e-01   3.8793e-02